<font size=4 face='黑体'>使用RNN进行MNIST数据集分类

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(r'F:\data\mnist_data\MNIST_data',one_hot=True)

H:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting F:\data\mnist_data\MNIST_data\train-images-idx3-ubyte.gz
Extracting F:\data\mnist_data\MNIST_data\train-labels-idx1-ubyte.gz
Extracting F:\data\mnist_data\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting F:\data\mnist_data\MNIST_data\t10k-labels-idx1-ubyte.gz


In [2]:
n_inputs = 28 #输入一行，一行有28个数据
max_time = 28 #一共28行
lstm_size = 100 #隐层单元
n_classes = 10 #10个分类
batch_size = 50
n_batch = mnist.train.num_examples // batch_size

x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
biases = tf.Variable(tf.constant(0.1,shape=[n_classes]))

#定义RNN网络
def RNN(X):
    inputs = tf.reshape(X,[-1,max_time,n_inputs])
    #定义LSTM基本CELL
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    #final_state[0]是cell_state
    #final_state[1]是hidden_state
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    results = tf.matmul(final_state[1],weights) + biases
    return results

results = RNN(x)
prediction = tf.nn.softmax(results)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=results))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

correct = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(6):
        for batch in range(n_batch):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_x,y:batch_y})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('Iter'+str(epoch)+' Testing Accuracy:'+ str(acc))

Iter0 Testing Accuracy:0.8338
Iter1 Testing Accuracy:0.8976
Iter2 Testing Accuracy:0.9196
Iter3 Testing Accuracy:0.9349
Iter4 Testing Accuracy:0.944
Iter5 Testing Accuracy:0.9502
